# DS_PROD-1. Подготовка модели к продакшену и деплой

###  Содержание <a class="anchor" id=0></a>

- [2. Сохранение и загрузка моделей: pickle и joblib](#2)

- [2.1 Библиотека Joblib](#3)

- [3. Практика: pickle](#4)

- [4. Сохранение и загрузка моделей: PMML и ONNX-ML](#5)

# 2. Сохранение и загрузка моделей: pickle и joblib <a class="anchor" id=2></a>

## 2.1 Библиотека pickle 


In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LinearRegression

In [20]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [21]:
# Serialization (сериализация)
model = pickle.dumps(regressor)

print(type(regressor))
print(type(model))

<class 'sklearn.linear_model._base.LinearRegression'>
<class 'bytes'>


In [22]:
# De-Serialization (десериализация)

regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

In [23]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

In [24]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

In [25]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
print(all(regressor.predict(X) == regressor_from_bytes.predict(X)))
## True
print(all(regressor.predict(X) == regressor_from_file.predict(X)))
## True

True
True


## ОГРАНИЧЕНИЯ

Как мы упоминали, у pickle есть ограничения. Например, мы не можем сериализовать лямбда-функции. Давайте посмотрим, что нам вернёт следующий код:

In [ ]:
my_lambda = lambda x: x*2
with open('my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

Совет. В таких случаях лучше пользоваться пакетом [dill](https://github.com/uqfoundation/dill).

In [26]:
from dill import dumps, loads

squared = lambda x: x**2
loads(dumps(squared))(3)

9

## Pipelines

In [19]:
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001BF53058F70>)),
                ('Linear', LinearRegression())])

In [27]:
# Сериализуем pipeline и записываем результат в файл
with open('my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [28]:
# Десериализуем pipeline из файла
with open('my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [29]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


>Примечание. Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции `dumps()` и `loads()`, а не `dump()` и `load()`.

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами `sklearn`. Например, для решения многих задач в нашем курсе мы часто использовали `feature engineering`, чтобы повысить качество работы моделей. Как встроить этот шаг в исходный пайплайн?

Для этого в `sklearn` можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: `TransformerMixin` и `BaseEstimator`.

Посмотрим на шаблон кастомного трансформера:

In [30]:
from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

У трансформера должно быть три обязательных метода:

* `__init__()` — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков `PolynomialFeatures` из `sklearn` в методе `__init__()` параметр `degree` задаёт степень полинома.

* `fit()` — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (`self`).
Например, в трансформере `StandardScaler` в методе `fit()` прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода `fit()`.

* `transform()` — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода `transform()` у `StandardScaler` из `sklearn` внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе `fit()`.

>Примечание. Как мы знаем, у некоторых трансформеров из `sklearn`, например у того же `MinMaxScaler`, есть ещё и метод `fit_transform()`, который является комбинацией методов `fit()` и `transform()`.

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе `transform()` эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы `__init__()` и `fit()` остаются без изменений.

In [31]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

Посмотрим, как работает наш кастомный трансформер. Создадим объект трансформера, вызовем метод transform и посмотрим на результирующий размер таблицы.

In [32]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь давайте встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [33]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001BF53058F70>)),
                ('Linear', LinearRegression())])

Наконец можно сериализовать полученный `pipeline`:

In [34]:
# Сериализуем pipeline и записываем результат в файл
with open('my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [36]:
# Задание 2.5

# Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. 
# Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

# Десериализуем pipeline из файла
with open('my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)
    
loaded_pipe.predict(features)

array([173.01985747])

# Библиотке Joblib  <a class="anchor" id=3></a>

[к содержанию](#0)

Как мы видим, `pickle` прекрасно справляется со своей задачей: мы можем сериализовать и восстанавливать любые `Python`-объекты, включая модели и даже пайплайны. Однако иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из `pickle` невозможно восстановить объект полностью.

В таких случаях вместо `pickle` лучше использовать библиотеку `joblib`. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — `.joblib`.

В остальном работа с `joblib` полностью идентична работе с `pickle`: после обучения модели производим сериализацию с помощью функции `dump()`, а в коде самого приложения, где нужно использовать модель, выполняем десериализацию с помощью функции `load()`. В каждую из этих функций необходимо передать путь до файла для записи и чтения соответственно.

Для иллюстрации работы сохраним полученную линейную регрессию:

In [38]:
import joblib

import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'regr.joblib')

## ['regr.joblib']

['regr.joblib']

In [39]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

# 3. Практика: pickle <a class="anchor" id=4></a>

[к содержанию](#0)



In [2]:
# Задание 3.1 При загрузке вывелся секретный код. Введите его в поле ниже.

import pickle

with open('model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)
    
model

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


c:\Users\AubakirovMA\AppData\Local\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


LinearRegression(positive=True)

In [4]:
# Задание 3.3 Теперь необходимо применить модель. Сделайте предсказание для следующего набора фичей: [1, 1, 1, 0.661212487096872]. 
# Введите результат, предварительно округлив его до трёх знаков после точки-разделителя.

model.predict([[1, 1, 1, 0.661212487096872]])

array([0.666])

In [10]:
# У присланной вам модели есть два поля (атрибута) с именами a и b. Создайте из них словарь с такими же именами ключей и значениями, а затем сохраните его в файл с помощью модуля pickle.

my_dict = {'a': model.a, 'b': model.b}

# Сериализуем pipeline и записываем результат в файл
with open('my_dict.pkl', 'wb') as output:
    pickle.dump(my_dict, output)

In [11]:
!python hw1_check_ol.py my_dict.pkl

('secret code 2:', '3c508')


# 4. Сохранение и загрузка моделей: PMML и ONNX-ML <a class="anchor" id=5></a>

[к содержанию](#0)


>Среда или требования к инференсу модели для вашего проекта могут быть устроены так, что потребуют реализации на языке программирования, отличном от `Python`. Например, если компания разрабатывает десктопное приложение, то для внедрения модели её потребуется «перевести» на `Java` или `C++`. Как это сделать?

## PREDICTIVE MODEL MARKUP LANGUAGE

В таких случаях используется генерация файла формата `PMML` (`Predictive Model Markup Language`).

`PMML` — это `XML`-диалект, который применяется для описания статистических и `DS`-моделей. `PMML`-совместимые приложения позволяют легко обмениваться моделями данных между собой. Разработка и внедрение `PMML` осуществляется IT-консорциумом `Data Mining Group`.

Подробнее с `PMML` можно ознакомиться на [официальном сайте](https://dmg.org/pmml/v4-4/GeneralStructure.html).

К сожалению, далеко не все библиотеки для машинного обучения (в том числе `sklearn`) поддерживают возможность сохранения обученной модели в указанном формате. Однако для этого можно использовать сторонние библиотеки, и одной из самых популярных является [Nyoka](https://nyoka-pmml.github.io/nyoka/).

Давайте сохраним модель из предыдущего блока в формат `PMML`.

Для установки можно использовать систему управления пакетами `pip`: `pip install nyoka`

In [12]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

Итак, мы построили пайплайн обработки данных и обучили модель линейной регрессии. После этого мы с помощью функции `skl_to_pmml` сохранили модель в файл `pipe.pmml`.

Откройте файл `pipe.pmml` с помощью любого текстового редактора.

Давайте рассмотрим этот файл подробнее:

* Секция `<DataDictionary>` содержит информацию о признаках, включая наименование и тип данных, используемых для построения модели.

* Секция `<TransformationDictionary>` содержит информацию о необходимых преобразованиях для каждого признака. Обратите внимание, что в этом блоке также содержится информация для трансформации. Так как мы использовали `minMaxScaler()`, то в файле записаны минимальное и максимальное значения.

***

# OPEN NEURAL NETWORK EXCHANGE

В разработке моделей на основе нейронных сетей сегодня наиболее распространён формат `ONNX` (`Open Neural Network Exchange`).

`ONNX` ([Open Neural Network Exchange](https://onnx.ai/)) — это открытый стандарт для обеспечения совместимости моделей машинного обучения. Он позволяет разработчикам искусственного интеллекта использовать модели с различными инфраструктурами, инструментами, средами исполнения и компиляторами.

Стандарт совместно поддерживается компаниями `Microsoft`, `Amazon`, `Facebook` и другими партнёрами как проект с открытым исходным кодом.

Часто стандарт `ONNX` и его библиотеки используют для конвертации из одного фреймворка в другой (например, из `PyTorch` в `TensorFlow` для использования в продакшене). Для конвертации различных фреймворков (не только DL) в формат ONNX и обратно существует [ряд библиотек](https://github.com/onnx):

* [ONNX-Tensorflow](https://github.com/onnx/onnx-tensorflow);
* [Tensorflow-ONNX](https://github.com/onnx/tensorflow-onnx);
* [Keras-ONNX](https://github.com/onnx/keras-onnx);
* [Sklearn-ONNX](https://github.com/onnx/sklearn-onnx).
* и другие.

Также в рамках стандарта `ONNX` есть инструмент [ONNX-runtime](https://github.com/microsoft/onnxruntime). Он служит для ускорения инференса `Python`-моделей, а также инференса на других языках, например `Java`, `C++`.

>pip install onnx
>
>pip install skl2onnx

>Для выполнения этого задания вам понадобится ознакомиться с документацией [здесь](http://onnx.ai/sklearn-onnx/api_summary.html) и [здесь](https://onnx.ai/sklearn-onnx/).

In [ ]:
import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import ___1___
from skl2onnx.common.data_types import ___2___


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(___3___, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(___4___)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', ___5___([None, ___6___]))]
model_onnx = ___7___(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.___8___("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)

In [28]:
from sklearn.linear_model import LogisticRegression
import numpy as np

X = np.array([['one', 2, 3], [3, 2, 1], [4, 5, 6]])
y = np.array([1, 2, 3])

LogisticRegression().fit(X, y)

ValueError: could not convert string to float: 'one'